# 모델
- LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct

# 단계별 전체 흐름
---
1. 각 파일별로 텍스트 chunk → 벡터DB(FAISS 등) 저장
---
2. 질문 입력
    - → ex) "두피에 하얗게 벗겨지고 가려워요. 무슨 질환일까요?"
---
3. 벡터DB에서 chunk 검색(Top-k 유사도 검색)
    - → 질문 임베딩 → 벡터DB에서 가장 유사한 chunk 반환
---
4. 검색 결과와 사용자 질문을 EXAONE 모델로 전달
    - → Prompt에 ‘검색 결과 + 원 질문’ 조합하여 입력
---
5. EXAONE이 최종 답변 생성
---

# 4번 셀 중 4-5 진행하면 됩니다.
    - 프롬프트 수정중

# 1. 임포트 & 데이터 폴더 지정

In [7]:
# 데이터 처리 및 파일 입출력
import os
import pandas as pd
import glob

# RAG, 임베딩, 벡터DB 관련 예시 임포트 (필요에 따라 사용)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM

# 텍스트 전처리, 토큰화 등 (필요에 따라)
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import re

# 마크다운 글씨 굵게 하기
from IPython.display import display, Markdown

print(torch.cuda.is_available())
print(torch.cuda.device_count())

# 파일 경로 자동 지정
data = "../dataset"

True
1


# 2. dataset 폴더 내 모든 CSV 파일 읽어오기

In [2]:
df_1200 = pd.read_csv(os.path.join(data, "1200_v1.csv"))
df_amc = pd.read_csv(os.path.join(data, "amc.csv"))
df_daily = pd.read_csv(os.path.join(data, "daily_dataset.csv"))
df_final = pd.read_csv(os.path.join(data, "final_v7.csv"))
df_kdca = pd.read_csv(os.path.join(data, "kdca.csv"))
df_snu = pd.read_csv(os.path.join(data, "snu.csv"))

# 데이터 미리보기 (각 파일별)
print("1200_v1.csv:", df_1200.shape, "\n", df_1200.head(2), "\n")
print("amc.csv:", df_amc.shape, "\n", df_amc.head(2), "\n")
print("daily_dataset.csv:", df_daily.shape, "\n", df_daily.head(2), "\n")
print("final_v7.csv:", df_final.shape, "\n", df_final.head(2), "\n")
print("kdca.csv:", df_kdca.shape, "\n", df_kdca.head(2), "\n")
print("snu.csv:", df_snu.shape, "\n", df_snu.head(2), "\n")

1200_v1.csv: (1200, 2) 
   label                                               text
0    건선  지난 몇 주 동안 팔, 다리, 몸통에 피부 발진이 생겼어요. 빨갛고 가려운데, 마른...
1    건선  제 피부가 벗겨지고 있어요, 특히 무릎, 팔꿈치, 그리고 두피 쪽이요. 이 벗겨짐이... 

amc.csv: (1278, 7) 
                                 병명  \
0   18번 염색체 단완결실 증후군(18p monosomy)   
1  18번 염색체 장완결실 증후군 (18q monosomy)   

                                                  정의  \
0  18번 염색체 단완결실 증후군은 흔하지 않지만 특징적인 표현형을 가지고 있는 질환을...   
1  18번 염색체 장완결실 증후군은 흔한 염색체 이상 질환은 아니지만, 특징적인 표현형...   

                                                  원인  \
0  대부분 염색체 검사를 통해 18번 염색체 단완결실이 확인됩니다. 이 경우 환자의 8...   
1  18번 염색체 장완결실 증후군은 대부분 자연발생적으로 발생합니다. 나머지는 18번 ...   

                                                  증상  \
0  18번 염색체 단완결실 증후군에는 소두증, 안검 하수, 사시, 양안 격리증, 크고 ...   
1  18번 염색체 장완결실 증후군은 소두증, 안면 중앙부 저형성, 깊게 패인 눈, 짧은...   

                                                  진단  \
0  18번 염색체 단완결실 증후군은 일반적으로 세포 유전학적 검사(말초혈액 염색체 검사...   
1  18번 염색체 장완결실 증후군의 진단은 일반적으로 세포 유전학적 검사

# 3. 각 파일별로 chunking → 벡터DB 저장

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


def chunk_text(texts, chunk_size=300):
    """간단하게 문자열을 일정 길이로 자르는 함수"""
    chunks = []
    for text in texts:
        if pd.isnull(text):
            continue
        for i in range(0, len(text), chunk_size):
            chunk = text[i : i + chunk_size]
            if chunk.strip():
                chunks.append(chunk)
    return chunks


### 1. 1200_v1.csv
df_1200 = pd.read_csv(os.path.join(data, "1200_v1.csv"))
texts_1200 = df_1200["label"].astype(str) + "\n" + df_1200["text"].astype(str)
chunks_1200 = chunk_text(texts_1200.tolist(), chunk_size=300)
db_1200 = FAISS.from_texts(chunks_1200, embedding=embedding_model)
db_1200.save_local("faiss_db_1200_v1")

### 2. amc.csv
df_amc = pd.read_csv(os.path.join(data, "amc.csv"))
# '병명', '정의', '원인', '증상', '진단', '치료', '진료과' 모두 하나로 합치기 (필요한 부분만 골라도 됨)
amc_texts = (
    df_amc["병명"].astype(str)
    + "\n"
    + df_amc["정의"].astype(str)
    + "\n"
    + df_amc["원인"].astype(str)
    + "\n"
    + df_amc["증상"].astype(str)
    + "\n"
    + df_amc["진단"].astype(str)
    + "\n"
    + df_amc["치료"].astype(str)
)
chunks_amc = chunk_text(amc_texts.tolist(), chunk_size=400)
db_amc = FAISS.from_texts(chunks_amc, embedding=embedding_model)
db_amc.save_local("faiss_db_amc")

### 3. daily_dataset.csv
df_daily = pd.read_csv(os.path.join(data, "daily_dataset.csv"))
daily_texts = df_daily["증상"].astype(str) + "\n" + df_daily["일상말"].astype(str)
chunks_daily = chunk_text(daily_texts.tolist(), chunk_size=250)
db_daily = FAISS.from_texts(chunks_daily, embedding=embedding_model)
db_daily.save_local("faiss_db_daily_dataset")

### 4. final_v7.csv
df_final = pd.read_csv(os.path.join(data, "final_v7.csv"))
final_texts = df_final["label"].astype(str) + "\n" + df_final["text"].astype(str)
chunks_final = chunk_text(final_texts.tolist(), chunk_size=300)
db_final = FAISS.from_texts(chunks_final, embedding=embedding_model)
db_final.save_local("faiss_db_final_v7")

### 5. kdca.csv
df_kdca = pd.read_csv(os.path.join(data, "kdca.csv"))
kdca_texts = (
    df_kdca["병명"].astype(str)
    + "\n"
    + df_kdca["정의"].astype(str)
    + "\n"
    + df_kdca["원인"].astype(str)
    + "\n"
    + df_kdca["증상"].astype(str)
    + "\n"
    + df_kdca["진단"].astype(str)
    + "\n"
    + df_kdca["치료"].astype(str)
)
chunks_kdca = chunk_text(kdca_texts.tolist(), chunk_size=400)
db_kdca = FAISS.from_texts(chunks_kdca, embedding=embedding_model)
db_kdca.save_local("faiss_db_kdca")

### 6. snu.csv
df_snu = pd.read_csv(os.path.join(data, "snu.csv"))
snu_texts = (
    df_snu["병명"].astype(str)
    + "\n"
    + df_snu["정의"].astype(str)
    + "\n"
    + df_snu["원인"].astype(str)
    + "\n"
    + df_snu["증상"].astype(str)
    + "\n"
    + df_snu["진단/검사"].astype(str)
    + "\n"
    + df_snu["치료"].astype(str)
)
chunks_snu = chunk_text(snu_texts.tolist(), chunk_size=400)
db_snu = FAISS.from_texts(chunks_snu, embedding=embedding_model)
db_snu.save_local("faiss_db_snu")

c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttenti

# 4-1. RAG(검색+생성) 파이프라인으로 작업
- 초기 답변 코드

## 문제점
1. ** 글 **으로 출력
2. 답변 속도 느림
3. 답변이 길게 출력

In [ ]:
# # [1] 사용자 질문 입력
# user_question = input("질문을 입력하세요: ")

# # [2] 여러 벡터DB 모두 로드
# db_paths = [
#     "faiss_db_1200_v1",
#     "faiss_db_amc",
#     "faiss_db_daily_dataset",
#     "faiss_db_final_v7",
#     "faiss_db_kdca",
#     "faiss_db_snu",
# ]
# db_list = [
#     FAISS.load_local(db_path, embedding_model, allow_dangerous_deserialization=True)
#     for db_path in db_paths
# ]

# # [3] 각 DB에서 Top-k 검색, 결과 합치기
# top_k = 2
# retrieved_chunks = []
# for db in db_list:
#     docs = db.similarity_search(user_question, k=top_k)
#     retrieved_chunks.extend([doc.page_content for doc in docs])

# # [4] context(검색 결과) 텍스트로 결합
# retrieved_context = "\n---\n".join(retrieved_chunks)

# # [5] 프롬프트 생성 (질문 부분 제거, 3가지 항목 요구)
# prompt = f"""
# [검색된 환자 사례]
# {retrieved_context}

# [답변 지침]
# 아래 3가지 항목을 반드시 각 2문장 이내, 전체 8줄 이내로 요약해서 번호로 구분해 답변하세요.

# 1. 예상되는 병명 2~3가지를 적고, 각각 한 문장으로 설명하세요.
# 2. 진료가 필요한 경우, 추천 진료과를 한글로 명확히 1줄로 써주세요.
# 3. 예방 및 관리 방법을 한글로 2줄 이내로 설명하세요.
# """

# # [6] EXAONE LLM 로드 (GPU 자동 할당)
# model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name, trust_remote_code=True, device_map="auto"
# )

# # [7] 입력 텐서도 GPU로 올리기
# inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
# inputs = {k: v.to(model.device) for k, v in inputs.items()}

# # [8] 답변 생성
# outputs = model.generate(**inputs, max_new_tokens=512)
# answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

# print("\n[EXAONE 답변]\n", answer)

# 4-5. RAG(검색+생성) 파이프라인으로 작업

- 프롬프트 수정 중

In [ ]:
# [1] 사용자 질문 입력
user_question = input("질문을 입력하세요: ")

# [2] 여러 벡터DB 모두 로드
db_paths = [
    "faiss_db_1200_v1",
    "faiss_db_amc",
    "faiss_db_daily_dataset",
    "faiss_db_final_v7",
    "faiss_db_kdca",
    "faiss_db_snu",
]
db_list = [
    FAISS.load_local(db_path, embedding_model, allow_dangerous_deserialization=True)
    for db_path in db_paths
]

# [3] 각 DB에서 Top-k 검색, 결과 합치기 (속도↑, top_k=1)
top_k = 1
retrieved_chunks = []
for db in db_list:
    docs = db.similarity_search(user_question, k=top_k)
    retrieved_chunks.extend([doc.page_content for doc in docs])

# [4] context(검색 결과) 텍스트로 결합 (전체 길이 제한)
retrieved_context = "\n---\n".join(retrieved_chunks)
max_context_length = 600
retrieved_context = retrieved_context[:max_context_length]

# [5] 프롬프트 생성 (딱 3문장만, 안내/예시 금지)
prompt = f"""
{retrieved_context}

아래 환자 증상에 대해 딱 3개 항목만 답변하세요.
각 항목을 반드시 아래 형식으로, 번호(1,2,3)와 콜론(:) 오른쪽에만 한글로 답변하세요.
절대 줄 바꿈, 4번 이상 번호, 안내문, 예시, 설명, 반복은 출력하지 마세요.
1. 예상되는 병명(2~3가지):
2. 추천 진료과(1줄):
3. 예방 및 관리 방법(2줄 이내):
"""

# [6] EXAONE LLM 로드 (GPU, FP16 권장)
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",  # GPU 자동 할당
    torch_dtype=torch.float16,  # FP16 (지원시)
)

# [7] 입력 텐서도 GPU로 강제 올리기
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}  # GPU 강제 이동
else:
    print("CUDA를 사용할 수 없습니다. CPU로 실행합니다.")

# [8] 답변 생성 (max_new_tokens 축소)
outputs = model.generate(**inputs, max_new_tokens=200)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

# [9] 번호 답변만 추출 (정규표현식)
answer_only = "\n".join(re.findall(r"^[1-3]\..*", answer, flags=re.MULTILINE))

print("\n[EXAONE 답변]\n")
display(Markdown(answer_only))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



[EXAONE 답변]



1. 예상되는 병명(2~3가지)과 각각 한 문장 설명
2. 추천 진료과(1줄)
3. 예방 및 관리 방법(2줄 이내)
1. **감기**: 바이러스 감염으로 인한 호흡기 증상으로 기침이 가장 흔한 증상 중 하나입니다.
2. **폐렴**: 폐에 염증이 생겨 기침이 심해지고 호흡곤란이 동반될 수 있습니다.
3. **위식도 역류 질환**: 위산이 식도로 역류하여 기침을 유발하며, 가슴 통증과 함께 나타날 수 있습니다.

In [ ]:
prompt = f"""
{retrieved_context}

아래 세 가지 항목에 대해 반드시 1,2,3번을 모두 빠짐없이 번호로 구분해서 한글로 답변하세요. 
각 항목이 누락되지 않게 반드시 출력하세요.  
불필요한 설명, 안내, 예시는 절대 출력하지 마세요.

예상되는 병명 2~3가지를 적고, 각각 한 문장으로 설명하세요.
진료가 필요한 경우, 추천 진료과를 한글로 명확히 1줄로 써주세요.
예방 및 관리 방법을 한글로 2줄 이내로 설명하세요.
"""